In [1]:
import pandas as pd
import numpy as np
import re
import string

from collections import Counter
from nltk.corpus import stopwords

from keras_preprocessing import text

from itertools import groupby

from tqdm.autonotebook import tqdm

pd.set_option('display.max_colwidth', -1)

/Users/etienne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  del sys.path[0]


In [2]:
pos_df = pd.read_csv("Datasets/twitter-datasets/train_pos_full.txt", delimiter="\\n", header=None, 
                     names = ["tweets"], engine='python')
neg_df = pd.read_csv("Datasets/twitter-datasets/train_neg_full.txt", delimiter="\\n", header=None,
                     names = ["tweets"], engine='python')

In [3]:
test_df = pd.read_csv("Datasets/twitter-datasets/test_data.txt", delimiter="\\n", header=None,
                     names = ["tweets"], engine='python')
test_df = pd.DataFrame(test_df.tweets.str.split(',',1).tolist(), columns = ['id','tweets'])

In [4]:
pos_df['label'] = 1
neg_df['label'] = 0

Remove numbers

In [5]:
pos_df.tweets = pos_df.tweets.str.replace('\d+', '')
neg_df.tweets = neg_df.tweets.str.replace('\d+', '')
test_df.tweets = test_df.tweets.str.replace('\d+', '')

Remove `<user>`

In [6]:
pos_df.tweets = pos_df.tweets.str.replace("<user>", "")
neg_df.tweets = neg_df.tweets.str.replace("<user>", "")
test_df.tweets = test_df.tweets.str.replace("<user>", "")

Remove `<url>`

In [7]:
pos_df.tweets = pos_df.tweets.str.replace("<url>", "")
neg_df.tweets = neg_df.tweets.str.replace("<url>", "")
test_df.tweets = test_df.tweets.str.replace("<url>", "")

In [8]:
pos_df.head()

,tweets,label
0,"i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe",1
1,"because your logic is so dumb , i won't even crop out your name or your photo . tsk .",1
2,""" just put casper in a box ! "" looved the battle ! #crakkbitch",1
3,thanks sir > > don't trip lil mama ... just keep doin ya thang !,1
4,visiting my brother tmr is the bestest birthday gift eveerrr ! ! !,1


Remove punctuation

In [9]:
pos_df.tweets = pos_df.tweets.str.translate(str.maketrans('', '', string.punctuation.replace('#','')))
neg_df.tweets = neg_df.tweets.str.translate(str.maketrans('', '', string.punctuation.replace('#','')))
test_df.tweets = test_df.tweets.str.translate(str.maketrans('', '', string.punctuation.replace('#','')))

Remove consecutive spaces

In [10]:
pos_df.tweets = pos_df.tweets.apply(lambda t : " ".join(t.split()))
neg_df.tweets = neg_df.tweets.apply(lambda t : " ".join(t.split()))
test_df.tweets = test_df.tweets.apply(lambda t : " ".join(t.split()))

In [11]:
pos_df.head()

,tweets,label
0,i dunno justin read my mention or not only justin and god knows about that but i hope you will follow me #believe,1
1,because your logic is so dumb i wont even crop out your name or your photo tsk,1
2,just put casper in a box looved the battle #crakkbitch,1
3,thanks sir dont trip lil mama just keep doin ya thang,1
4,visiting my brother tmr is the bestest birthday gift eveerrr,1


In [12]:
results_pos = Counter()
pos_df.tweets.str.lower().str.split().apply(results_pos.update)
results_neg = Counter()
neg_df.tweets.str.lower().str.split().apply(results_neg.update)

0          None
1          None
2          None
3          None
4          None
           ... 
1249995    None
1249996    None
1249997    None
1249998    None
1249999    None
Name: tweets, Length: 1250000, dtype: object

In [13]:
count_pos = results_pos.most_common(5000)
count_neg = results_neg.most_common(5000)

dict_count_pos = dict(count_pos)
dict_count_neg = dict(count_neg)

both = dict_count_pos.keys() & (dict_count_neg.keys())

count_total = [(word, dict_count_pos[word]+dict_count_neg[word]) for word in both]
dict_both = dict(count_total)

In [14]:
pos_90 = [(word, dict_count_pos[word]) for word in both if 100*dict_count_pos[word]/dict_both[word]>90]
neg_90 = [(word, dict_count_neg[word]) for word in both if 100*dict_count_neg[word]/dict_both[word]>90]

In [15]:
pos_90

[('thank', 36670),
 ('thanks', 62337),
 ('thankyou', 4418),
 ('goodmorning', 2540),
 ('shoutout', 6033),
 ('congrats', 3488)]

In [16]:
neg_90

[('memory', 9868),
 ('black', 32352),
 ('series', 14385),
 ('pc', 8871),
 ('complete', 31269),
 ('battery', 10358),
 ('throat', 2566),
 ('cap', 1862),
 ('hurts', 5617),
 ('health', 6076),
 ('technology', 2499),
 ('metal', 1687),
 ('classic', 5530),
 ('professional', 1794),
 ('market', 3650),
 ('inch', 9441),
 ('screw', 2974),
 ('poster', 26372),
 ('pro', 4083),
 ('collection', 5022),
 ('dvd', 14668),
 ('japanese', 1964),
 ('screen', 5161),
 ('mobile', 2899),
 ('cherry', 2532),
 ('sports', 4703),
 ('mm', 5315),
 ('system', 3996),
 ('wide', 27148),
 ('brand', 8899),
 ('cried', 3409),
 ('cell', 3775),
 ('guide', 9125),
 ('ap', 1891),
 ('digital', 6435),
 ('mb', 6945),
 ('gb', 11486),
 ('reference', 1467),
 ('headache', 2831),
 ('edition', 22579),
 ('design', 4414),
 ('kindle', 9346),
 ('cd', 13578),
 ('ca', 1672),
 ('volume', 4172),
 ('laptop', 8801),
 ('sad', 24209),
 ('size', 6600),
 ('oz', 6958),
 ('silver', 5361),
 ('print', 3650),
 ('wood', 13521),
 ('studies', 2349),
 ('co', 2252),


In [17]:
pos_80_90 = [(word, dict_count_pos[word]) for word in both if ((100*dict_count_pos[word]/dict_both[word]>80) & (100*dict_count_pos[word]/dict_both[word]<=90))]
neg_80_90 = [(word, dict_count_neg[word]) for word in both if ((100*dict_count_neg[word]/dict_both[word]>80) & (100*dict_count_neg[word]/dict_both[word]<=90))]

In [18]:
pos_80_90

[('vote', 2328),
 ('awesome', 10923),
 ('appreciate', 2498),
 ('enjoyed', 1279),
 ('shout', 3782),
 ('glad', 8093),
 ('wonderful', 2835),
 ('proud', 5693),
 ('directioners', 1553),
 ('follower', 2275),
 ('happy', 31629),
 ('hehe', 6099),
 ('excited', 8302),
 ('retweet', 5151),
 ('goodnight', 4806),
 ('hey', 22355),
 ('smile', 10834),
 ('xoxo', 2003),
 ('lovely', 5848),
 ('goood', 1553),
 ('followback', 1326),
 ('hehehe', 1303),
 ('birthday', 19452),
 ('following', 13919),
 ('yay', 5265),
 ('kissing', 1254),
 ('welcome', 7195),
 ('spread', 1138),
 ('pussy', 1322)]

In [19]:
neg_80_90

[('bk', 2011),
 ('blackberry', 1206),
 ('dark', 4805),
 ('network', 1507),
 ('calendar', 662),
 ('wall', 3695),
 ('soft', 2103),
 ('ouch', 779),
 ('wah', 1480),
 ('feat', 847),
 ('natural', 3455),
 ('adventure', 1379),
 ('wars', 920),
 ('windows', 1179),
 ('mini', 2602),
 ('debut', 822),
 ('card', 4040),
 ('hurting', 1377),
 ('smooth', 2239),
 ('heavy', 1666),
 ('tummy', 1243),
 ('control', 2098),
 ('bags', 1990),
 ('pe', 828),
 ('shoulder', 1616),
 ('edge', 1133),
 ('law', 2049),
 ('university', 1658),
 ('ipod', 5810),
 ('machine', 1304),
 ('river', 1334),
 ('popular', 1744),
 ('library', 2999),
 ('american', 5003),
 ('net', 716),
 ('fo', 1144),
 ('comfort', 1120),
 ('messed', 737),
 ('tape', 1472),
 ('wind', 1210),
 ('computer', 3021),
 ('multiple', 982),
 ('sales', 951),
 ('stock', 935),
 ('security', 1348),
 ('cold', 6532),
 ('poor', 5416),
 ('shoe', 1051),
 ('pound', 850),
 ('art', 4301),
 ('bottles', 1079),
 ('box', 4395),
 ('winter', 924),
 ('oil', 1656),
 ('however', 1621),
 ('

In [20]:
pos_70_80 = [(word, dict_count_pos[word]) for word in both if ((100*dict_count_pos[word]/dict_both[word]>70) & (100*dict_count_pos[word]/dict_both[word]<80))]
neg_70_80 = [(word, dict_count_neg[word]) for word in both if ((100*dict_count_neg[word]/dict_both[word]>70) & (100*dict_count_neg[word]/dict_both[word]<=80))]

In [21]:
pos_70_80

[('aha', 2861),
 ('beautiful', 14209),
 ('bitches', 2005),
 ('hah', 830),
 ('alright', 3473),
 ('beliebers', 1004),
 ('hows', 2075),
 ('sharing', 936),
 ('indeed', 1090),
 ('smell', 1602),
 ('bday', 3439),
 ('sweet', 7498),
 ('youre', 26646),
 ('xx', 24525),
 ('#thoughtsduringschool', 1447),
 ('selena', 1203),
 ('hahaha', 12380),
 ('liam', 6405),
 ('shall', 2601),
 ('finally', 8067),
 ('luck', 6047),
 ('interesting', 1540),
 ('freak', 782),
 ('surprise', 1183),
 ('info', 1275),
 ('unless', 1491),
 ('avi', 1953),
 ('gorgeous', 2645),
 ('heres', 2389),
 ('pleasure', 1069),
 ('yesss', 1856),
 ('love', 90924),
 ('jk', 2297),
 ('nice', 18831),
 ('kay', 720),
 ('weed', 1424),
 ('great', 22588),
 ('hun', 3071),
 ('good', 72422),
 ('follows', 2273),
 ('conversations', 783),
 ('yeahhh', 792),
 ('zayn', 2964),
 ('singer', 1307),
 ('kisses', 960),
 ('lips', 1226),
 ('lookin', 959),
 ('umm', 681),
 ('yours', 4015),
 ('lots', 3287),
 ('mmm', 1589),
 ('positive', 1191),
 ('lemme', 752),
 ('worry', 4

In [22]:
positive = pos_70_80 + pos_90
negative = neg_70_80 + neg_90

In [23]:
positive

[('aha', 2861),
 ('beautiful', 14209),
 ('bitches', 2005),
 ('hah', 830),
 ('alright', 3473),
 ('beliebers', 1004),
 ('hows', 2075),
 ('sharing', 936),
 ('indeed', 1090),
 ('smell', 1602),
 ('bday', 3439),
 ('sweet', 7498),
 ('youre', 26646),
 ('xx', 24525),
 ('#thoughtsduringschool', 1447),
 ('selena', 1203),
 ('hahaha', 12380),
 ('liam', 6405),
 ('shall', 2601),
 ('finally', 8067),
 ('luck', 6047),
 ('interesting', 1540),
 ('freak', 782),
 ('surprise', 1183),
 ('info', 1275),
 ('unless', 1491),
 ('avi', 1953),
 ('gorgeous', 2645),
 ('heres', 2389),
 ('pleasure', 1069),
 ('yesss', 1856),
 ('love', 90924),
 ('jk', 2297),
 ('nice', 18831),
 ('kay', 720),
 ('weed', 1424),
 ('great', 22588),
 ('hun', 3071),
 ('good', 72422),
 ('follows', 2273),
 ('conversations', 783),
 ('yeahhh', 792),
 ('zayn', 2964),
 ('singer', 1307),
 ('kisses', 960),
 ('lips', 1226),
 ('lookin', 959),
 ('umm', 681),
 ('yours', 4015),
 ('lots', 3287),
 ('mmm', 1589),
 ('positive', 1191),
 ('lemme', 752),
 ('worry', 4

In [24]:
positive = [i[0] for i in positive]
negative = [i[0] for i in negative]


In [25]:
positive

['aha',
 'beautiful',
 'bitches',
 'hah',
 'alright',
 'beliebers',
 'hows',
 'sharing',
 'indeed',
 'smell',
 'bday',
 'sweet',
 'youre',
 'xx',
 '#thoughtsduringschool',
 'selena',
 'hahaha',
 'liam',
 'shall',
 'finally',
 'luck',
 'interesting',
 'freak',
 'surprise',
 'info',
 'unless',
 'avi',
 'gorgeous',
 'heres',
 'pleasure',
 'yesss',
 'love',
 'jk',
 'nice',
 'kay',
 'weed',
 'great',
 'hun',
 'good',
 'follows',
 'conversations',
 'yeahhh',
 'zayn',
 'singer',
 'kisses',
 'lips',
 'lookin',
 'umm',
 'yours',
 'lots',
 'mmm',
 'positive',
 'lemme',
 'worry',
 'followers',
 'btw',
 'ahaha',
 'ooh',
 'funny',
 '#teamfollowback',
 'sure',
 'youu',
 'anytime',
 'cool',
 'cute',
 'relax',
 'mention',
 'marry',
 'luv',
 'whoever',
 'sweetie',
 'handsome',
 'trending',
 'checking',
 'ahah',
 'ha',
 'wishes',
 'check',
 'loving',
 'fav',
 'amazing',
 'hahah',
 'cutest',
 'agree',
 'ladies',
 'alll',
 'niggas',
 'yall',
 'surf',
 'trend',
 'fantastic',
 'chilling',
 'pretty',
 'yep',

In [26]:
dic = {}
for el in positive:
    dic[el] = 'happy'
    
for el in negative:
    dic[el] = 'sad'

In [27]:
#pos_df.tweet_as_list = pos_df.tweet_as_list.apply(lambda tweet_list: [dic(word) if word in dic.keys else word for word i])

pos_df.tweets = pos_df.tweets.apply(lambda tweet: " ".join([dic[word] if word in dic.keys() else word for word in tweet.split(" ")]))
                                    
neg_df.tweets = neg_df.tweets.apply(lambda tweet: " ".join([dic[word] if word in dic.keys() else word for word in tweet.split(" ")]))
                                    
test_df.tweets = test_df.tweets.apply(lambda tweet: " ".join([dic[word] if word in dic.keys() else word for word in tweet.split(" ")]))
                                    
                                    

In [61]:
np.savetxt("Datasets/twitter-datasets/train_pos_full_cleaned.txt", pos_df.values, fmt='%s')
np.savetxt("Datasets/twitter-datasets/train_neg_full_cleaned.txt", neg_df.values, fmt='%s')

In [28]:
pos_df.to_csv("Datasets/twitter-datasets/train_pos_full_cleaned_2.csv")
neg_df.to_csv("Datasets/twitter-datasets/train_neg_full_cleaned_2.csv")
test_df.to_csv("Datasets/twitter-datasets/test_data_cleaned_2.csv")